## 10.Convolution Neural Network
- convolution이 어떤 연산인지
- MNIST에서 CNN만들어보기
- Pytorch Visdom
- Pytorch Datasets & custom dataset
- CIFAR-10
- VGG & ResNet

### 10-1 Convolution
- convolution: 이미지 위에서 stride 값 만큼 filter를 이동시키면서 겹쳐지는 부분의 각 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 연산
- stride: filter를 한번에 얼마나 이동할 것인가?
- padding: 이미지 상하좌우에 0으로 된 띠가 둘러진다.
----
- torch.nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias) 코드를 이용.
- kernel_size: 필터 사이즈, 3X3 이면 그냥 3이라고 쓴다.
- input type은 torch.Tensor, input shape는 (batch_size, channel, height, width)
----
- output size =(input size - filter size + 2*padding)/stride +1
    - 예제1) 55 예제2) 29 예제3) 32 예제4) 28x60 예제5)64x32
    - 예제2처럼 소숫점이 나오는 경우 버림하면 된다.

In [28]:
import torch
import torch.nn as nn
#예제 1번을 확인해보자.
conv=nn.Conv2d(1,1,11,4,0)
conv

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))

In [29]:
inputs=torch.Tensor(1,1,227,227)

In [30]:
conv(inputs).shape
# 이렇게 우리가 위에서 계산한 내용을 확인할 수 있다.

torch.Size([1, 1, 55, 55])

In [31]:
conv=nn.Conv2d(1,1,7,2,0)
conv
inputs=torch.Tensor(1,1,64,64)
conv(inputs).shape
# 예제 2번도 확인 가능하다.

torch.Size([1, 1, 29, 29])

---
- Neuron(perceptron)과 Convolution
    - 우리가 지금까지 배운 convolution filter가 perceptron의 weight라고 생각할 수 있다. 
    - perceptron에서는 weight와 bias가 존재했으므로 convolution에도 bias가 있다.
---
- Pooling
    - max pooling(2)는 2x2 사이즈에서 가장 큰 숫자를 뱉어냄.
    - avarage pooling은 평균을 뱉어냄.
    - MaxPool2d(kernel_size)
----
- CNN implementation

In [32]:
input=torch.Tensor(1,1,28,28)
conv1=nn.Conv2d(1,5,5)
pool=nn.MaxPool2d(2)
out=conv1(input)
out2=pool(out)
print(out.size())
print(out2.size())

torch.Size([1, 5, 24, 24])
torch.Size([1, 5, 12, 12])


- convolution 연산과 pooling 연산을 배웠다.
- 다음 시간에는 MNIST dataset에 CNN을 적용하자.
---
- One more thing
    - filter를 뒤집고 연산 convolution, 그대로 연산 cross-correlation 

### 10-2. MNIST CNN
- 딥러닝을 학습시키는 단계
    - 1. 라이브러리 가져오기
    - 2. GPU 사용 설정하고 random value를 위한 seed 설정
    - 3. 학습에 사용되는 parameter 설정
    - 4. 데이터셋 가져오고 loader 만들기
    - 5. 학습 모델 만들기 (class CNN(torch.nn.Module))
    - 6. loss function 선택하고 최적화도구 선택
    - 7. 모델 학습 및 loss check
    - 8. 학습된 모델의 성능을 확인한다.
   

In [33]:
input=torch.Tensor(1,1,28,28)
conv1=nn.Conv2d(1,32,3,padding=1)
pool=nn.MaxPool2d(2)
conv2=nn.Conv2d(32,64,3,padding=1)

In [34]:
out=conv1(input)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [35]:
out2=pool(out)
print(out2.shape)

torch.Size([1, 32, 14, 14])


In [36]:
out3=conv2(out2)
print(out3.shape)

torch.Size([1, 64, 14, 14])


In [37]:
out4=pool(out3)
print(out4.shape)

torch.Size([1, 64, 7, 7])


In [38]:
out5=out4.view(out4.size(0),-1)
#배치사이즈는 그대로 두고 나머지는 한 줄로 펼쳐서 넣어라
out.shape

torch.Size([1, 32, 28, 28])

In [39]:
fc=nn.Linear(3136,10)
out6= fc(out5)
out6
out6.shape

torch.Size([1, 10])

- 연습은 여기까지 하고 실제로 만들어보자.

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:

#GPU설정, cuda가 이용 가능할때 cuda로 하고 아닐때는 cpu사용.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility 랜덤 value를 고정해주는 코드
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
print(device)

cpu


In [43]:
learning_rate=0.001
training_epochs=15
batch_size=100

In [44]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                       train=True,
                       transform=transforms.ToTensor(),
                       download=True)
#토치비전, 데이터셋에 있는 데이터를 불러오자

mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

In [45]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True)

In [47]:
#CNN 모델을 만들어보자.
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        # 레이어 쌓는 중 
        
        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.fc=torch.nn.Linear(7*7*64,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)#초기화 진행
        
    def forward(self,x):
        out=self.layer1(x)
        out2=self.layer2(out)
        out3=out2.view(out.size(0),-1)
        out4=self.fc(out3)
        return out4

In [48]:
model=CNN().to(device)

In [49]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [50]:
total_batch=len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost+=cost/total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))
    
print('Learning FInished!')

Learning started. It takes sometime.


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1)==Y_test
    accuracy=correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

- 레이어를 더 깊게 쌓으면 더 정확도가 높을까?

In [ ]:
#CNN 모델을 만들어보자.
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        # 레이어 쌓는 중 
        
        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        
    
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out
    
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
total_batch=len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost+=cost/total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))
    
print('Learning FInished!')

In [ ]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

- 깊게 레이어를 쌓으면 오히려 더 정확도가 낮다.
- 무작정 깊게 쌓는 것보다 효율성이 더 중요하다.